In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

from cleantext import clean
import hazm

df = pd.read_excel('/content/drive/MyDrive/Data.xlsx')
data = df[~(df['descpishnahad'].duplicated(keep = False))]
Data = data[pd.notnull(data['descpishnahad'])]
Data.head()

 df = pd.DataFrame(Data['descpishnahad'])
 df
df['descpishnahad_len_by_words'] = df['descpishnahad'].astype(str).apply(lambda t: len(hazm.word_tokenize(t)))
df = df.dropna(subset=['descpishnahad_len_by_words'])
df = df.reset_index(drop=True)
import re
def cleaning(text):
    text = text.strip()

    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )


    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)

    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)

    text = wierd_pattern.sub(r'', text)

    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)

    return text
df['cleaned_descpishnahad'] = df['descpishnahad'].astype(str).apply(cleaning)


# calculate the length of descpishnahad based on their words
df['cleaned_descpishnahad_len_by_words'] = df['cleaned_descpishnahad'].apply(lambda t: len(hazm.word_tokenize(t)))
Df = pd.DataFrame(df['cleaned_descpishnahad'])
Df.columns = ['descpishnahad']
Df.head()

stop_words = hazm.stopwords_list(stopwords_file='/usr/local/lib/python3.6/dist-packages/hazm/data/stopwords.dat')

def remove_stopwords(text):
   tokens = []
   for token in text.split():
       if token not in stop_words:
          tokens.append(token)
   return " ".join(tokens)

Df['cleaned_descpishnahad'] = Df['descpishnahad'].apply(lambda x: remove_stopwords(x))

Df['cleaned_descpishnahad'] = Df['cleaned_descpishnahad'].apply(cleaning)
from sentence_transformers import models, SentenceTransformer, util
from sklearn.cluster import KMeans

corpus = list(Df['cleaned_descpishnahad'])
def load_st_model(model_name_or_path):
    word_embedding_model = models.Transformer(model_name_or_path)
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False)

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    return model


# Load the Sentence-Transformer
embedder = load_st_model('m3hrdadfi/bert-fa-base-uncased-farstail-mean-tokens')
corpus_embeddings = embedder.encode(corpus, show_progress_bar=True)


import keras
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam

m = Sequential()
m.add(Dense(512,  activation='relu', input_shape=(768,)))
m.add(Dense(128,  activation='relu'))
m.add(Dense(64,  activation='relu'))
m.add(Dense(32,  activation='relu'))
m.add(Dense(2,    activation='linear', name="bottleneck"))
m.add(Dense(32,  activation='relu'))
m.add(Dense(64,  activation='relu'))
m.add(Dense(128,  activation='relu'))
m.add(Dense(512,  activation='relu'))
m.add(Dense(768,  activation='sigmoid'))
m.compile(loss='mean_squared_error', optimizer = Adam())
history = m.fit(corpus_embeddings, corpus_embeddings, batch_size=128, epochs=10, verbose=1)

encoder = Model(m.input, m.get_layer('bottleneck').output)
Zenc = encoder.predict(corpus_embeddings)
Renc = m.predict(corpus_embeddings)

kmeans = KMeans(n_clusters=3, n_init=50 , random_state=1)
kmeans.fit(Renc)
score = silhouette_score(Renc, km.labels_, metric='euclidean')
score
